In [6]:
import numpy as np
from matplotlib import pyplot as plt

from dftpy.ions import Ions
from dftpy.functional import Functional
from dftpy.formats import io

from ase.io import read
from ase.lattice.spacegroup import Spacegroup
from ase.lattice.spacegroup import crystal

from scipy.optimize import minimize

from dftpy.ions import Ions
from dftpy.functional import Functional
from dftpy.formats import io
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional, ExternalPotential
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.optimization import Optimization
import ase

In [3]:
with open('he.out') as D:
    k = [match for match in D if "!    total energy" in match]
KS_te = str(k).split()[4]

In [13]:
path_pp='/Users/valeria/Documents/PP/psl/'
file1='He.pz-rrkjus_psl.1.0.0.UPF'
PP_list = {'He': path_pp+file1}

In [17]:
XC = Functional(type='XC',name='LDA')
HARTREE = Functional(type='HARTREE')

In [14]:
def min_energy(x0, *args):
    KS_TE, rho, ions = args
    KE = Functional(type='KEDF',name='WT', rho0=x0)
    grid = DirectGrid(lattice=ions.cell, nr=[len(rho),len(rho),len(rho[0,0])])
    PSEUDO = LocalPseudo(grid = grid, ions=ions, PP_list=PP_list, rcut=10)
    rho_ini = DirectField(grid=grid)
    rho_ini[:] = ions.get_ncharges()/ions.cell.volume

    evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)
    optimization_options = {'econv' : 1e-5*ions.nat}
    opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options, 
                       optimization_method = 'TN')
    rho = opt.optimize_rho(guess_rho=rho_ini)
    OF_TE = evaluator.Energy(rho=rho, ions=ions)
    # OF_KE = KE(rho).energy
    diff = np.abs(KS_TE*1/2-OF_TE)
    print(diff)
    return diff

In [18]:
rho = io.read_density('he.xsf')
inputfile = 'he.vasp'
ions = Ions.from_ase(ase.io.read(inputfile, format='vasp'))
KS_TE = float(KS_te)
minn = minimize(min_energy, 0.01, args = (KS_TE, rho, ions), method='Powell', bounds=[[0,1]], options={'ftol' : 1e-6})
    

setting key: He -> /Users/valeria/Documents/PP/psl/He.pz-rrkjus_psl.1.0.0.UPF
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -4.120160008539E-01     -4.120160E-01   3.435016E-02    1       1       4.720592E-01    
!WARN: Change to steepest decent
1       -1.923759773954E+00     -1.511744E+00   1.752554E+00    1       5       1.538461E+00    
2       -2.781413825060E+00     -8.576541E-01   3.148558E-01    25      2       6.050992E+00    
3       -2.807897784725E+00     -2.648396E-02   7.069155E-02    12      1       8.476692E+00    
4       -2.813005158307E+00     -5.107374E-03   6.173324E-03    17      1       1.182694E+01    
5       -2.813261135519E+00     -2.559772E-04   9.340682E-04    14      1       1.428523E+01    
6       -2.813354564936E+00     -9.342942E-05   8.739159E-05    20      1       1.760356E+01    
7       -2.813360410063E+00     -5.845127E-06   7.639412E-06    16      1       2.046719E+01    
8       -2.81336

#### Density Optimization Converged ####
Chemical potential (a.u.): -0.14004545291935405
Chemical potential (eV)  : -3.8108308803458915
1.1492068445168115
setting key: He -> /Users/valeria/Documents/PP/psl/He.pz-rrkjus_psl.1.0.0.UPF
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -4.120160008539E-01     -4.120160E-01   3.435016E-02    1       1       3.251216E-01    
!WARN: Change to steepest decent
1       -1.604261692181E+00     -1.192246E+00   1.194411E+00    1       4       1.226890E+00    
2       -1.669089364201E+00     -6.482767E-02   9.053526E-01    14      1       3.584237E+00    
3       -1.814572684622E+00     -1.454833E-01   2.390780E-01    16      2       6.305571E+00    
4       -1.820812502720E+00     -6.239818E-03   3.966685E-02    13      1       8.311365E+00    
5       -1.821019352747E+00     -2.068500E-04   2.642468E-02    13      1       1.039100E+01    
6       -1.821156578545E+00     -1.372258E-04   2.19979

Chemical potential (eV)  : -16.776315134776553
0.1310424730833457
setting key: He -> /Users/valeria/Documents/PP/psl/He.pz-rrkjus_psl.1.0.0.UPF
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -4.120160008539E-01     -4.120160E-01   3.435016E-02    1       1       2.972870E-01    
!WARN: Change to steepest decent
1       -1.899710680552E+00     -1.487695E+00   1.732624E+00    1       5       1.517481E+00    
2       -2.037691640430E+00     -1.379810E-01   3.078001E+00    14      1       4.933174E+00    
3       -2.126031158015E+00     -8.833952E-02   2.411042E+00    13      1       7.518107E+00    
4       -2.744015435756E+00     -6.179843E-01   5.740184E-01    14      3       1.052789E+01    
5       -2.775109693913E+00     -3.109426E-02   5.779194E-02    13      1       1.272276E+01    
6       -2.783547139811E+00     -8.437446E-03   4.296644E-03    18      1       1.604178E+01    
7       -2.783838996089E+00     -2.918563E-04  

Chemical potential (eV)  : -20.518438333715938
0.019305845099153895
setting key: He -> /Users/valeria/Documents/PP/psl/He.pz-rrkjus_psl.1.0.0.UPF
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -4.120160008539E-01     -4.120160E-01   3.435016E-02    1       1       3.753700E-01    
!WARN: Change to steepest decent
1       -1.943374754250E+00     -1.531359E+00   1.769779E+00    1       5       1.419604E+00    
2       -2.086054261176E+00     -1.426795E-01   3.288415E+00    14      1       4.205353E+00    
3       -2.176016965956E+00     -8.996270E-02   2.370669E+00    13      1       6.606189E+00    
4       -2.816970887899E+00     -6.409539E-01   1.259114E-01    18      3       1.015558E+01    
5       -2.835936381058E+00     -1.896549E-02   6.621618E-02    12      1       1.256500E+01    
6       -2.837727874386E+00     -1.791493E-03   5.566447E-03    9       1       1.430765E+01    
7       -2.838236248596E+00     -5.083742E-04

In [19]:
minn.fun, minn.x

(0.018973827401350007, array([0.00806902]))

In [20]:
np.save('rho_ENE', minn.x)